In [ ]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 21.8 MB/s eta 0:00:00


In [ ]:
import networkx as nx
import random
import heapq
from typing import Tuple, Set, Dict, List

def ler_instancia(nome_arquivo: str) -> Tuple[nx.DiGraph, int, int, int, List[Tuple[int, int]]]:
    """
    Lê a instância do problema a partir de um arquivo e retorna:
      - G: grafo direcionado com nós (x,y) para uma grade de 1 a n.
      - delta: atraso adicional para nós efetivamente protegidos.
      - T: tempo limite para a propagação.
      - alpha: número total de recursos disponíveis (soma dos k dos grupos).
      - grupos: lista de tuplas (t, k) com os tempos de disponibilidade e quantidade de recursos.

    Formato do arquivo:
      Linha 1: n delta T
      Linha 2: m
      Linhas 3 a 2+m: t k    (no tempo t, k recursos disponíveis)
      Linhas seguintes: x1-y1 x2-y2 t   (arco de (x1,y1) a (x2,y2) com tempo t)
    """
    with open(nome_arquivo, 'r') as f:
        linhas = [linha.strip() for linha in f if linha.strip()]

    # Linha 1: n delta T
    partes = linhas[0].split()
    n = int(partes[0])
    delta = int(partes[1])
    T = int(partes[2])

    # Cria os nós da grade: (1,1) até (n,n)
    G = nx.DiGraph()
    for x in range(1, n + 1):
        for y in range(1, n + 1):
            G.add_node((x, y))

    # Linha 2: m (número de grupos de recursos)
    m = int(linhas[1])
    alpha = 0
    grupos: List[Tuple[int, int]] = []
    for i in range(2, 2 + m):
        t_str, k_str = linhas[i].split()
        t = int(t_str)
        k = int(k_str)
        grupos.append((t, k))
        alpha += k

    # Linhas seguintes: arcos no formato "x1-y1 x2-y2 t"
    for linha in linhas[2 + m:]:
        parte1, parte2, tempo_str = linha.split()
        t_arco = int(tempo_str)
        x1_str, y1_str = parte1.split('-')
        x2_str, y2_str = parte2.split('-')
        u = (int(x1_str), int(y1_str))
        v = (int(x2_str), int(y2_str))
        # Se já houver uma aresta, mantém a de menor tempo (opcional)
        if G.has_edge(u, v):
            peso_atual = G[u][v].get('weight', float('inf'))
            if t_arco < peso_atual:
                G[u][v]['weight'] = t_arco
        else:
            G.add_edge(u, v, weight=t_arco)

    return G, delta, T, alpha, grupos

def dijkstra_propagation(G: nx.DiGraph, s: Tuple[int, int], protegidos: Set[Tuple[int, int]], delta: int) -> Dict[Tuple[int, int], int]:
    """
    Executa o algoritmo de Dijkstra para a propagação da fake news.
    Se o nó u estiver efetivamente protegido, o custo para sair de u é acrescido de delta.

    Parâmetros:
      - G: grafo com atributo 'weight' nas arestas.
      - s: nó semente (origem da propagação).
      - protegidos: conjunto de nós efetivamente protegidos (com atraso aplicado).
      - delta: atraso adicional para nós protegidos.

    Retorna:
      - dist: dicionário com o tempo de chegada (distância) de s para cada nó.
    """
    dist = {node: float('inf') for node in G.nodes()}
    dist[s] = 0
    heap = [(0, s)]

    while heap:
        d_u, u = heapq.heappop(heap)
        if d_u > dist[u]:
            continue
        # Se u estiver protegido, o custo de sair dele aumenta em delta.
        extra = delta if u in protegidos else 0
        for v in G.neighbors(u):
            w = G[u][v].get('weight', 1)
            nova_dist = dist[u] + w + extra
            if nova_dist < dist[v]:
                dist[v] = nova_dist
                heapq.heappush(heap, (nova_dist, v))
    return dist

def avaliar_solucao(G: nx.DiGraph, servidor_origem: Tuple[int, int], protegidos: Set[Tuple[int, int]],
                     delta: int, T: int, grupos_recursos: List[Tuple[int, int]]) -> int:
    """
    Avalia uma solução determinando quantos nós (servidores) são afetados
    pela fake news (ou seja, que são alcançados em tempo <= T).

    Para avaliar a proteção, fazemos:
      1. Calcula-se a propagação base sem proteção (delta = 0).
      2. Dentre os nós designados para proteção, somente aqueles que puderem
         receber o recurso a tempo (isto é, cujo tempo base de chegada é >= t de disponibilidade (β))
         serão efetivamente protegidos.
         Essa alocação é feita seguindo a ordem dos grupos (menor t primeiro)
         e, para cada grupo, atribuindo no máximo k nós.
      3. Com os nós efetivamente protegidos, recalcula-se a propagação (com delta).

    Retorna:
      - Número de servidores afetados (com tempo de chegada <= T).
    """
    # 1. Propagação base (sem proteção)
    dist_base = dijkstra_propagation(G, servidor_origem, protegidos=set(), delta=0)

    # 2. Alocação efetiva dos recursos: somente nós protegidos que forem alcançados
    # após o recurso estar disponível terão o atraso aplicado.
    # Ordena os grupos por tempo de disponibilidade (crescente)
    grupos_ordenados = sorted(grupos_recursos, key=lambda g: g[0])
    # Ordena os nós designados (em 'protegidos') pelo tempo base (quanto mais cedo a fake news chegaria, maior a urgência)
    candidatos = sorted(protegidos, key=lambda node: dist_base[node])
    servidores_com_recurso = set()

    for tempo_grupo, recursos_grupo in grupos_ordenados:
        count = 0
        for servidor in candidatos:
            # Se já recebeu recurso, pula
            if servidor in servidores_com_recurso:
                continue
            # Se o tempo base de chegada é maior ou igual ao tempo em que o recurso está disponível,
            # então a proteção pode ser efetiva.
            if dist_base[servidor] >= tempo_grupo:
                servidores_com_recurso.add(servidor)
                count += 1
                if count >= recursos_grupo:
                    break

    # 3. Propagação com proteção efetiva (aplicando delta somente nos nós com recurso efetivo)
    dist = dijkstra_propagation(G, servidor_origem, protegidos=servidores_com_recurso, delta=delta)
    num_afetados = sum(1 for d in dist.values() if d <= T)
    return num_afetados

    # Acha os servidores alcançáveis
    # Ele aloca os recursos nos servidores canditados à solução
    # Recalcula os servidores alcançáveis
    # Retorna o número de servidores afetados dessa solução

def vizinho_swap(solucao: Set[Tuple[int, int]], G: nx.DiGraph) -> Set[Tuple[int, int]]:
    """
    Gera um vizinho da solução realizando uma troca:
      - Remove um nó aleatório da solução.
      - Adiciona um nó aleatório que não estava na solução.

    A solução resultante mantém o mesmo tamanho.
    """
    solucao_nova = set(solucao)
    todos_nos = set(G.nodes())

    if solucao_nova:
        no_remover = random.choice(list(solucao_nova))
    else:
        no_remover = None

    # Pega todos os nós menos o da solução atual
    candidatos = list(todos_nos - solucao_nova)

    if not candidatos:
        return solucao_nova

    no_adicionar = random.choice(candidatos)

    if no_remover is not None:
        solucao_nova.remove(no_remover)

    solucao_nova.add(no_adicionar)

    return solucao_nova

    # Remove um nó dos servidores que terão recurso
    # Adiciona um nó aletório que não teria recurso

def melhor_vizinho(solucao: Set[Tuple[int, int]], G: nx.DiGraph, servidor_origem: Tuple[int, int],
                   delta: int, T: int, grupos: List[Tuple[int, int]], num_candidatos: int = 10) -> Tuple[Set[Tuple[int, int]], int]:
    """
    Gera um conjunto de vizinhos (por meio do operador swap) e retorna o melhor vizinho,
    isto é, aquele que minimiza o número de servidores afetados.
    """
    melhor_solucao = solucao
    numero_afetados_melhor_solucao = avaliar_solucao(G, servidor_origem, solucao, delta, T, grupos)

    for _ in range(num_candidatos):
        solucao_candidata = vizinho_swap(solucao, G)
        numero_afetados_solucao_candidata = avaliar_solucao(G, servidor_origem, solucao_candidata, delta, T, grupos)

        if numero_afetados_solucao_candidata < numero_afetados_melhor_solucao:
            melhor_solucao, numero_afetados_melhor_solucao = solucao_candidata, numero_afetados_solucao_candidata

    return melhor_solucao, numero_afetados_melhor_solucao

    # Vou trocar o vizinho por um aleatório um número "num_candidatos" de vezes
    # O que resultar em uma melhor solução é retornado

def perturbar_solucao(solucao: Set[Tuple[int, int]], G: nx.DiGraph, num_trocas: int) -> Set[Tuple[int, int]]:
    """
    Aplica uma perturbação à solução realizando 'num_trocas' trocas aleatórias.

    Ele realiza uma troca de candidatos à servidores protegidos um número "num_trocas" de vezes
    Como se fosse uma busca local mais intensa.
    """
    solucao_nova = set(solucao)
    todos_nos = set(G.nodes())

    for _ in range(num_trocas):
        if solucao_nova:
            no_remover = random.choice(list(solucao_nova))
        else:
            no_remover = None
        candidatos = list(todos_nos - solucao_nova)
        if candidatos:
            no_adicionar = random.choice(candidatos)
            if no_remover is not None:
                solucao_nova.remove(no_remover)
            solucao_nova.add(no_adicionar)

    return solucao_nova


def busca_local_iterada(G: nx.DiGraph, servidor_origem: Tuple[int, int], alpha: int, delta: int, T: int,
                          grupos_recursos: List[Tuple[int, int]], iter_max: int = 400, perturba_steps: int = 3) -> Tuple[Set[Tuple[int, int]], int]:
    """
    Executa a meta-heurística de Busca Local Iterada (ILS) para minimizar
    o número de servidores afetados.

    Parâmetros:
      - G: grafo.
      - servidor_origem: nó semente (origem da propagação).
      - alpha: número total de recursos disponíveis (tamanho da solução).
      - delta: atraso adicional para nós protegidos.
      - T: tempo limite para propagação.
      - grupos_recursos: lista de grupos de recursos (t, k).
      - iter_max: número máximo de iterações do ILS.
      - perturba_steps: número de trocas para a perturbação.

    Retorna:
      - melhor_solucao: melhor conjunto de nós protegidos encontrado.
      - numero_afetados_melhor_solucao: valor da função objetivo (número de servidores afetados) da melhor solução.
    """
    servidores = list(G.nodes())

    # Solução inicial: escolhe aleatoriamente 'alpha' nós para receber os recursos.
    solucao_atual = set(random.sample(servidores, alpha))
    melhor_solucao = set(solucao_atual)

    # Avalia a solução inicial:
    numero_afetados_melhor_solucao = avaliar_solucao(G, servidor_origem, solucao_atual, delta, T, grupos_recursos)

    iteracoes = 0
    while iteracoes < iter_max:
        # Busca local: tenta melhorar a solução atual via vizinhos
        solucao_vizinha, numero_afetados_solucao_vizinha = melhor_vizinho(solucao_atual, G, servidor_origem, delta, T, grupos_recursos, num_candidatos=10)
        numero_afetados_solucao_atual = avaliar_solucao(G, servidor_origem, solucao_atual, delta, T, grupos_recursos)

        if numero_afetados_solucao_vizinha < numero_afetados_solucao_atual:
            solucao_atual = solucao_vizinha
            if numero_afetados_solucao_vizinha < numero_afetados_melhor_solucao:
                numero_afetados_melhor_solucao = numero_afetados_solucao_vizinha
                melhor_solucao = set(solucao_vizinha)
                #print(f"Iteração {iteracoes}: Melhorou para {numero_afetados_melhor_solucao} servidores afetados.")
        else:
            # Se não houver melhoria local, aplica perturbação para sair de ótimos locais
            solucao_perturbada = perturbar_solucao(solucao_atual, G, perturba_steps)
            numero_afetados_solucao_perturbada = avaliar_solucao(G, servidor_origem, solucao_perturbada, delta, T, grupos_recursos)
            if numero_afetados_solucao_perturbada < numero_afetados_solucao_atual:
                solucao_atual = solucao_perturbada
                # Critério de aceitação ( Só aceita a nova solução se for melhor que a anterior ):
                if numero_afetados_solucao_perturbada < numero_afetados_melhor_solucao:
                    numero_afetados_melhor_solucao = numero_afetados_solucao_perturbada
                    melhor_solucao = set(solucao_perturbada)
                    #print(f"Perturbação iter {iteracoes}: Melhorou para {numero_afetados_melhor_solucao} servidores afetados.")
        iteracoes += 1
    return melhor_solucao, numero_afetados_melhor_solucao

def main():
    # Arquivo onde será inserido o resultado final da instância processada:
    nome_arquivo_resultado_final = input("Digite o nome do arquivo que será gravado o resultado final do algoritmo: ")
    input_iter_max = int(input("Digite o quantidade máxima de iterações: "))
    input_pertubacoes = int(input("Digite a quantidade de pertubações: "))
    arquivo_entrada = input("Digite o nome do arquivo de instância: ")

    # Parâmetro que influencia as escolhas do algoritmo
    random.seed(51)

    #print(f"\n=== Processando instância: {arquivo} ===")
    try:
      # Recebe os dados da instância e armazena nas variáveis do problema:
      G, delta, T, alpha, grupos_recursos = ler_instancia(arquivo_entrada)
      print("Parâmetros da Instância:")
      print(f"  Atraso adicional (delta): {delta}")
      print(f"  Tempo limite (T): {T}")
      print(f"  Número total de recursos (alpha): {alpha}")
      print(f"  Grupos de recursos: {grupos_recursos}")

      # Seleciona aleatoriamente qualquer nó do grafo para ser a semente (origem da propagação)
      servidor_origem = random.choice(list(G.nodes()))
      #print(f"Nó semente selecionado: {servidor_origem}")

      servidores_com_recurso, numero_servidores_afetados = busca_local_iterada(G, servidor_origem, alpha, delta, T, grupos_recursos, input_iter_max, input_pertubacoes)

      print("Resultado Final:")
      print("  Melhor conjunto de nós protegidos:")
      print(f"    {servidores_com_recurso}")
      print("  Número de servidores afetados (objetivo):", numero_servidores_afetados)
      print(numero_servidores_afetados, sep=" ")

      # Escrevendo solução encontrada no arquivo final
      with open(nome_arquivo_resultado_final, 'w') as f:
        f.write(str(numero_servidores_afetados))

      # Apagando estruturas de dados:
      del G, delta, T, alpha, grupos_recursos, servidor_origem, servidores_com_recurso, numero_servidores_afetados

    except Exception as e:
        print(f"Erro ao ler o arquivo {arquivo_entrada}: {e}")


if __name__ == '__main__':
    main()

Digite o nome do arquivo que será gravado o resultado final do algoritmo: a
Digite o quantidade máxima de iterações: 500
Digite a quantidade de pertubações: 3
Digite o nome do arquivo de instância: fn1.dat
Parâmetros da Instância:
  Atraso adicional (delta): 50
  Tempo limite (T): 70
  Número total de recursos (alpha): 12
  Grupos de recursos: [(10, 3), (20, 3), (30, 3), (40, 3)]
Resultado Final:
  Melhor conjunto de nós protegidos:
    {(4, 10), (14, 17), (11, 13), (6, 13), (3, 9), (7, 12), (8, 12), (9, 11), (4, 11), (11, 14), (10, 12), (5, 12)}
  Número de servidores afetados (objetivo): 233
233
